In [1]:
# 把函数视为对象
def factorial(n):
    """return n!"""
    return 1 if n < 2 else n * factorial(n - 1)

In [2]:
# __doc__是函数对象众多属性的一个
factorial.__doc__

'return n!'

In [3]:
type(factorial)

function

In [4]:
# 通过别的名称使用函数
fact = factorial
fact

<function __main__.factorial(n)>

In [5]:
fact(5)

120

In [6]:
# 函数可以作为参数传入另一个函数
map(factorial, range(11))

In [7]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [8]:
# 高阶函数
# 接受函数为参数，或者返回值为函数的函数就是高阶函数
# 内置函数sorted就是一个高阶函数，可选参数key可以接受一个函数对象
# 它会应用到各个元素上进行排序
# 根据单词的长度给列表排序
fruits = ['fig', 'apple', 'cherry', 'raspberry', 'banana', 'strawberry']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [9]:
# lambda - 匿名函数
# 在参数列表中最适合使用匿名函数
sorted(fruits, key=lambda x: x[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

In [10]:
# callable - 可调用对象
# 除用户定义的函数外，调用运算符（即()）还可以应用到其他对象上
# 可以使用callable()函数判断一个对象是否可以调用
# 以下7种对象是可调用对象：
# 1、用户定义的函数 - 使用def语句或lambda表达式创建
# 2、内置函数 - 使用CPyhton实现的函数
# 3、方法 - 在类定义体中定义的函数
# 4、类
# 5、类的实例
# 6、生成器函数
[callable(obj) for obj in [str, abs, 13]]

[True, True, False]

In [11]:
# 用户定义的可调用类型
# 只要实现实例方法__call__，任何对象都可以表现得像函数
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    # bingo.pick()的快捷方式就是bingo()
    def __call__(self):
        return self.pick()

In [12]:
bingo = BingoCage(range(3))
bingo.pick()

0

In [13]:
bingo()

2

In [14]:
callable(bingo)

True